In [26]:
import torch
import numpy as np
import torch.nn as nn
import sys
import os
project_root = os.path.abspath("..")  # Adjust if needed
import pytorch_lightning as pl
# Add the project root to sys.path
if project_root not in sys.path:
    sys.path.append(project_root)

from src.models.pointNetVae import PointNetVAE
from src.utils.data_utils import *
from src.dataset_classes.pointDataset import *
from proteinshake.datasets import ProteinFamilyDataset
from proteinshake.tasks import LigandAffinityTask
import random
from torch.utils.data import DataLoader, Dataset, Subset
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [49]:
dataset = ProteinFamilyDataset(root='../data').to_graph(eps = 8).pyg()
dataset = [data[0] for data in dataset]

In [19]:
graph = dataset[0][0]
leng = len(dataset[0][1]['protein']['sequence'])

In [72]:
dataset[0].x.unsqueeze(1).shape

torch.Size([277, 1])

In [98]:
from torch_geometric.loader import DataLoader

# dataset = [...]  # List of torch_geometric.data.Data objects (one per graph)
batch_size = 16
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
test_batch = next(iter(loader))

In [100]:
test_batch.edge_index

tensor([[   0,    0,    1,  ..., 3150, 3150, 3150],
        [   1,    2,    2,  ..., 3149, 2737, 2736]])

In [165]:
from torch_geometric.nn import GCNConv, dense_diff_pool, global_mean_pool, TopKPooling
hidden_dim = 32
latent_dim = 2
class encoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(1, hidden_dim, cached=True) # cached only for transductive learning
        self.conv2 = GCNConv(hidden_dim ,2*hidden_dim)

        self.fc_mu = nn.Linear(2*hidden_dim, latent_dim)
        self.fc_logvar = nn.Linear(2*hidden_dim, latent_dim)
        

    def forward(self,x):
        x_f, x_edg, x_batch = x.x.unsqueeze(-1).float(), x.edge_index, x.batch
        x_f = self.conv1(x_f, x_edg)

        x_f = self.conv2(x_f, x_edg)
        pooled_x = global_mean_pool(x_f, x_batch)
        mu = self.fc_mu(pooled_x)
        logvar = self.fc_logvar(pooled_x)
        return mu, logvar